## Loading Libraries

In [1]:
import pandas as pd
import numpy as np
import torch
import evaluate

c:\Users\Pawel\anaconda3\envs\env_torch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device('cuda')

## Loading Dataset

In [3]:
df = pd.read_csv('../data/processed/train_sample_processed.csv')

In [4]:
statuses = np.unique(df['OpenStatus'].values)

id2label = {idx: label for idx, label in enumerate(statuses)}
label2id = {label: idx for idx, label in enumerate(statuses)}

In [5]:

from custom_dataset import GithubDataset
from torch.utils.data import random_split
from datasets import Dataset

# Create the pytorch dataset
full_dataset = GithubDataset(df)

In [6]:
train_dataset, validation_dataset, test_dataset = random_split(full_dataset, [0.7, 0.25, 0.05])

train_dataset = Dataset.from_dict(train_dataset[:])
validation_dataset = Dataset.from_dict(validation_dataset[:])
test_dataset = Dataset.from_dict(test_dataset[:])

In [7]:
train_dataset = train_dataset.rename_columns({"text_content": "text", "status": "label"})
validation_dataset = validation_dataset.rename_columns({"text_content": "text", "status": "label"})
test_dataset = test_dataset.rename_columns({"text_content": "text", "status": "label"})

columns_to_remove = [
  'tags_onehot',
  'unrecognized_tags_count',
  'reputation',
  'undeleted_answers',
  'user_life_days',
  'title'
]

train_dataset = train_dataset.remove_columns(columns_to_remove)
validation_dataset = validation_dataset.remove_columns(columns_to_remove)
test_dataset = test_dataset.remove_columns(columns_to_remove)

### Tokenizing Data

In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [9]:
MAX_TEXT_CONTENT = 128

def tokenize_func(batch):
  tokenized_batch = tokenizer(batch['text'], padding=True, truncation=True, max_length=MAX_TEXT_CONTENT, return_tensors="pt")
  tokenized_batch["labels"] = [label2id[label] for label in batch["label"]]
  return tokenized_batch

In [10]:
tokenized_train_dataset = train_dataset.map(tokenize_func, batched=True)
tokenized_validation_dataset = validation_dataset.map(tokenize_func, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_func, batched=True)

Map: 100%|██████████| 7013/7013 [00:01<00:00, 6762.76 examples/s]


#### COMMENT - IDEAS

We probably should:
- retrain the whole model (probably smaller) with
- better tokenizer - built up from the ground including all the names of the specific tech (languages, frameworks, IDEs, etc.)

## Model - Custom

In [11]:
from custom_model import AutoCompositeModel

model = AutoCompositeModel(device).to(device)

In [12]:
# model.my_new_layers[0].weight.dtype

In [13]:
input = torch.tensor(tokenized_test_dataset['input_ids'][:10]).float().to(device)
input.dtype

torch.float32

## Training Setup

### Data Loaders

In [14]:
from torch.utils.data import DataLoader
training_loader = DataLoader(tokenized_train_dataset, batch_size=16, shuffle=True)
validation_loader = DataLoader(tokenized_validation_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(tokenized_test_dataset, batch_size=16, shuffle=True)

### Loss Function

In [15]:
loss_fn = torch.nn.CrossEntropyLoss()

### Optimizer

In [16]:
# Optimizers specified in the torch.optim package
# optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)

optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, weight_decay=1e-4)

In [17]:
from torchmetrics import Accuracy

accuracy_metric = Accuracy(task='multiclass', num_classes=5).to(device)

### Trainer Setup

In [18]:
from training_own import Trainer, TrainerConfiguration, get_model_params

config = TrainerConfiguration(
  training_loader=training_loader,
  validation_loader=validation_loader,
  optimizer=optimizer,
  loss_fn=loss_fn,
  accuracy_metric=accuracy_metric,
  device=device
)

In [19]:
get_model_params(model)

273445

In [20]:
trainer = Trainer(model=model, trainer_configuration=config, input_column='input_ids', output_column='labels')

In [21]:
len(training_loader)

6137

In [22]:
trainer.train_one_epoch(logging_frequency=10, evaluate_when_logging=True)

c:\Users\Pawel\Documents\_projects\Github-Questions-Classification\notebooks\training_own.py:58: UserWarning: Anomaly Detection has been enabled. This mode will increase the runtime and should only be enabled for debugging.
  with autograd.detect_anomaly():


 batch 10 training_loss: 1.6415608882904054 validation_loss: 1.6444809436798096 training_accuracy: 0.08749999850988388 validation_accuracy 0.07392411679029465
 batch 20 training_loss: 1.6561653733253479 validation_loss: 1.6445494890213013 training_accuracy: 0.03750000149011612 validation_accuracy 0.07564438879489899


KeyboardInterrupt: 